In [1]:
# !wget https://raw.githubusercontent.com/mesolitica/malaysian-dataset/master/llm-benchmark/BM-pt3/BM-A-pt3

import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
#mesolitica/malaysian-llama2-7b-32k-instructions-v2
tokenizer = AutoTokenizer.from_pretrained('mesolitica/malaysian-mistral-7b-32k-instructions')
# model = AutoModelForCausalLM.from_pretrained(
#     'fpf-7b-instructions-16k-call-v2/checkpoint-9300', 
#     use_flash_attention_2 = True, 
#     torch_dtype = torch.float16,
#     device_map="cuda:0"
# )

model = AutoModelForCausalLM.from_pretrained(
    'mesolitica/malaysian-mistral-7b-32k-instructions', 
    #use_flash_attention_2 = True, 
    torch_dtype = torch.float16,
    device_map="auto"
)

/home/cybertron/anaconda3/envs/sgpt/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████████████| 3/3 [00:04<00:00,  1.57s/it]


In [3]:
from tqdm import tqdm
import os
import random
import json

In [4]:

data_folder = '/media/cybertron/fa54fcb6-b5e1-492e-978a-6389519c168a/llm-benchmarks/sejarah-spm/data/'
with open(data_folder+ 'sejarah_spm') as fopen:
    text = fopen.read()
    
questions = []
for t in text.split('no: ')[1:]: # split by "no"
    t = t.strip()
    no = t.split('\n')[0] # "split by newline to get index'
    soalan = t.split('soalan:')[1].split('jawapan:')[0].strip()
    jawapan = t.split('jawapan: ')[1].split(',')[0].strip()
    data = {
        'no': no,
        'soalan': soalan,
        'jawapan': jawapan,
    }
    questions.append(data)

In [5]:
arange = set(range(len(questions)))

In [6]:
def convert_prompt(row, answer = False): # convert question to prompt 
    if answer: # include answer in prompt
        prompt = f""" 
soalan: {row['soalan']}
jawapan: {row['jawapan']}
    """
    else: # do not include answer in prompt
        prompt = f"""
soalan: {row['soalan']}
jawapan:
    """
    return prompt.strip()

def parse_mistral_chat(messages):

    user_query = messages[-1]['content']

    users, assistants = [], []
    for q in messages[:-1]:
        if q['role'] == 'user':
            users.append(q['content'])
        elif q['role'] == 'assistant':
            assistants.append(q['content'])

    texts = ['<s>']
    for u, a in zip(users, assistants):
        texts.append(f'[INST] {u.strip()} [/INST]{a.strip()}</s> ')

    texts.append(f'[INST] {user_query.strip()} [/INST]')
    prompt = ''.join(texts).strip()
    return prompt

In [7]:
for i in tqdm(range(len(questions))):
    
    shots = random.sample(arange - {i}, 1)
    prompts = []
    for no, s in enumerate(shots):
        prompts.append(f'Contoh soalan {no + 1}\n' + convert_prompt(questions[s], answer = True))

    prompts.append(convert_prompt(questions[i]))
    prompt = '\n\n'.join(prompts)
    messages = [
        {'role': 'user', 'content': prompt}
    ]
    prompt = parse_mistral_chat(messages)
    # print(prompt)
    # break
    inputs = tokenizer([prompt], return_tensors='pt', add_special_tokens=False).to('cuda')
    repeat = []
    for _ in range(5):
        try:
            generate_kwargs = dict(
                inputs,
                max_new_tokens=50,
                top_p=0.95,
                top_k=50,
                temperature=0.9,
                do_sample=True,
                num_beams=1,
                repetition_penalty=1.05,
            )
            r = model.generate(**generate_kwargs)
            splitted = tokenizer.decode(r[0]).split('[/INST]')[1].strip().replace('</s>', '').replace('.', '').replace(',', '').strip().split()
            splitted = [t for t in splitted if len(t) == 1 and t in {'A', 'B', 'C', 'D'}]
            repeat.append(splitted[0])
    
        except Exception as e:
            pass
    
    questions[i]['output'] = repeat

  0%|                                                    | 0/20 [00:00<?, ?it/s]/tmp/ipykernel_12128/328704830.py:3: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  shots = random.sample(arange - {i}, 1)
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
  5%|██▏                                         | 1/20 [00:03<01:01,  3.24s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_

In [8]:
def most_common(l):
    return max(set(l), key=l.count)

In [9]:
filtered = [q for q in questions if 'output' in q and len(q['output'])]
correct = 0
for q in filtered:
    correct += most_common(q['output']) == q['jawapan']
(correct / len(filtered)) * 100

50.0

In [10]:
for q in questions:
    print(q['output'], q['jawapan'])

['C', 'C', 'C', 'C', 'C'] C
['A', 'D', 'C', 'C', 'A'] A
['B', 'D', 'A', 'C', 'C'] C
['C', 'C', 'C', 'C', 'A'] A
['C', 'B', 'C', 'B', 'C'] C
['D', 'D', 'A', 'A', 'B'] C
['B', 'D', 'D', 'B', 'C'] C
['C', 'D', 'C', 'D', 'C'] A
['B', 'C', 'C', 'C', 'C'] B
['C', 'C', 'C', 'C', 'D'] A
['B', 'B', 'A', 'D', 'B'] B
['A', 'D', 'D', 'D', 'D'] A
['A', 'A', 'A', 'C', 'B'] A
['B', 'D', 'B', 'A', 'C'] B
['C', 'C', 'D', 'C', 'C'] D
['D', 'B', 'D', 'D', 'D'] D
['B', 'D', 'C', 'D', 'D'] A
['B', 'A', 'B', 'C', 'C'] B
['D', 'A', 'A', 'B', 'A'] A
['B', 'B', 'A', 'C', 'A'] B
